In [ ]:
import torch
from pathlib import Path

label_shards = [
    Path("../data/sample/labels/labels_shard_000.pt"),
    Path("../data/sample/labels/labels_shard_001.pt"),
    Path("../data/sample/labels/labels_shard_002.pt"),
    Path("../data/sample/labels/labels_shard_003.pt")
]

In [ ]:
pos, neg = 0, 0
for p in label_shards:
    shard = torch.load(p, map_location="cpu", weights_only=False)
    labels = shard["labels"]

    for _, y_full in labels.items():
        if y_full.dim() == 2 and y_full.size(1) == 3:
            # shape (L,3) --> [yes, uncertain, not]
            yes_col = y_full[:, 0].float()
            unc_col = y_full[:, 1].float()
            not_col = y_full[:, 2].float()

            mask = (unc_col == 0)
            pos += int(yes_col[mask].sum().item())
            neg += int(not_col[mask].sum().item())
        else:
            # bianry labels
            pos += int((y_full == 1).sum().item())
            neg += int((y_full == 0).sum().item())

print(f"num_pos: {pos}\nnum_neg: {neg}\npos_weight: {neg / max(pos, 1)}")